# Creating a digital twin
This notebook contains sample [EDSL](https://docs.expectedparrot.com/) code for creating an AI agent and prompting it to critique some content. The code is readily editable to create other agents and survey questions with any available language models. 

EDSL is an open-source library for simulating surveys and experiements with AI. Please see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started.

In [1]:
from edsl import (
    QuestionMultipleChoice,
    QuestionCheckBox,
    QuestionFreeText,
    QuestionLinearScale,
    QuestionList,
    QuestionBudget,
    Agent, 
    ScenarioList,
    Survey, 
    Model
)

In [2]:
# Construct relevant traits as a dictionary
agent_traits = {
    "persona": """You are a middle-aged mom in Cambridge, Massachusetts. 
        You hope to own a driverless minivan in the near future.
        You are working on an open source Python package for conducting research with AI.""",
    "age": 45,
    "location": "US",
    "industry": "information technology",
    "company": "Expected Parrot",
    "occupation": "startup cofounder",
    "hobbies": "kayaking, beach walks",
}

# Pass the traits and an optional name to an agent
agent = Agent(name="Robin", traits=agent_traits)

In [3]:
# Optionally create some special instructions for the task
context = """You are answering questions about a software package for conducting surveys and experiments 
          with large language models. The creators of the software want to know your opinions about some
          new features they are considering building. Your feedback will help them make decisions about
          those potential features. """

In [4]:
# Construct questions for the task
q1 = QuestionMultipleChoice(
    question_name="use_often",
    question_text=context
    + """Consider the following new feature: {{ content }}
    How often do you think you would use it?""",
    question_options=["Never", "Occasionally", "Frequently", "All the time"],
)

q2 = QuestionCheckBox(
    question_name="checkbox",
    question_text=context
    + """Consider the following new feature: {{ content }}
    Select all that apply.""",
    question_options=[
        "This feature would be useful to me.",
        "This feature would make me more productive.",
        "This feature will be important to me.",
        "The benefits of this feature are not clear to me.",
        "I would like to see some examples of how to use this feature.",
    ],
)

q3 = QuestionFreeText(
    question_name="concerns",
    question_text=context
    + "Do you have any concerns about the value and usefulness of this new feature: {{ content }}",
)

q4 = QuestionLinearScale(
    question_name="likely_to_use",
    question_text=context
    + """Consider the following new feature: {{ content }}
    On a scale from 1 to 5, how likely are you to use this new feature? 
    (1 = not at all likely, 5 = very likely)""",
    question_options=[1, 2, 3, 4, 5],
    option_labels={1: "Not at all likely", 5: "Very likely"},
)

In [5]:
# Create a survey with the questions
survey = Survey(questions=[q1, q2, q3, q4])

In [6]:
# Create some content for the agent to review
contents = [
    "An optional progress bar that shows how many of your questions have been answered while your survey is running.",
    "A method that lets you quickly check what version of the package you have installed.",
    "A method that lets you include questions and responses as context for new questions.",
]

# Parameterize the questions with the content
scenarios = ScenarioList.from_list("content", contents)

In [7]:
agent

Agent(name = """Robin""", traits = {'persona': 'You are a middle-aged mom in Cambridge, Massachusetts. \n        You hope to own a driverless minivan in the near future.\n        You are working on an open source Python package for conducting research with AI.', 'age': 45, 'location': 'US', 'industry': 'information technology', 'company': 'Expected Parrot', 'occupation': 'startup cofounder', 'hobbies': 'kayaking, beach walks'})

In [8]:
survey.to_scenario_list().print()

question_name,question_text,question_options,question_type,min_selections,option_labels,max_selections
use_often,You are answering questions about a software package for conducting surveys and experiments with large language models. The creators of the software want to know your opinions about some new features they are considering building. Your feedback will help them make decisions about those potential features. Consider the following new feature: {{ content }} How often do you think you would use it?,"['Never', 'Occasionally', 'Frequently', 'All the time']",multiple_choice,None,None,None
checkbox,You are answering questions about a software package for conducting surveys and experiments with large language models. The creators of the software want to know your opinions about some new features they are considering building. Your feedback will help them make decisions about those potential features. Consider the following new feature: {{ content }} Select all that apply.,"['This feature would be useful to me.', 'This feature would make me more productive.', 'This feature will be important to me.', 'The benefits of this feature are not clear to me.', 'I would like to see some examples of how to use this feature.']",checkbox,None,None,None
concerns,You are answering questions about a software package for conducting surveys and experiments with large language models. The creators of the software want to know your opinions about some new features they are considering building. Your feedback will help them make decisions about those potential features. Do you have any concerns about the value and usefulness of this new feature: {{ content }},None,free_text,None,None,None
likely_to_use,"You are answering questions about a software package for conducting surveys and experiments with large language models. The creators of the software want to know your opinions about some new features they are considering building. Your feedback will help them make decisions about those potential features. Consider the following new feature: {{ content }} On a scale from 1 to 5, how likely are you to use this new feature? (1 = not at all likely, 5 = very likely)","[1, 2, 3, 4, 5]",linear_scale,None,"{1: 'Not at all likely', 5: 'Very likely'}",None


In [9]:
# Run the survey and store the results, and show a progress bar
results = survey.by(scenarios).by(agent).run(progress_bar=True)

Output()

In [10]:
# Show all columns of the Results object
results.columns

['agent.age',
 'agent.agent_instruction',
 'agent.agent_name',
 'agent.company',
 'agent.hobbies',
 'agent.industry',
 'agent.location',
 'agent.occupation',
 'agent.persona',
 'answer.checkbox',
 'answer.concerns',
 'answer.likely_to_use',
 'answer.use_often',
 'comment.k_comment',
 'generated_tokens.checkbox_generated_tokens',
 'generated_tokens.concerns_generated_tokens',
 'generated_tokens.likely_to_use_generated_tokens',
 'generated_tokens.use_often_generated_tokens',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.checkbox_system_prompt',
 'prompt.checkbox_user_prompt',
 'prompt.concerns_system_prompt',
 'prompt.concerns_user_prompt',
 'prompt.likely_to_use_system_prompt',
 'prompt.likely_to_use_user_prompt',
 'prompt.use_often_system_prompt',
 'prompt.use_often_user_prompt',
 'question_options.checkbox_question_options',
 'q

In [11]:
# Print the responses
results.select(
    "content",
    "use_often",
    "checkbox",
    "concerns",
    "likely_to_use",
).print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ scenario                  ┃ answer     ┃ answer                    ┃ answer                    ┃ answer         ┃
┃ .content                  ┃ .use_often ┃ .checkbox                 ┃ .concerns                 ┃ .likely_to_use ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ A method that lets you    │ Frequently │ ['This feature would be   │ Oh, I think having a      │ 4              │
│ quickly check what        │            │ useful to me.', 'This     │ method to quickly check   │                │
│ version of the package    │            │ feature would make me     │ the version of the        │                │
│ you have installed.       │            │ more productive.', 'This  │ package installed is      │                │
│                           │            │ feature will be important │ definitely valuable and   │                │
│                           │            │ to me.']                  │ useful. When you're       │                │
│                           │            │                           │ working on research,      │                │
│                           │            │                           │ especially with AI and    │                │
│                           │            │                           │ large language models,    │                │
│                           │            │                           │ it's crucial to know      │                │
│                           │            │                           │ exactly which version     │                │
│                           │            │                           │ you're using. This can    │                │
│                           │            │                           │ help in replicating       │                │
│                           │            │                           │ results, troubleshooting  │                │
│                           │            │                           │ issues, and ensuring      │                │
│                           │            │                           │ compatibility with other  │                │
│                           │            │                           │ tools or datasets.        │                │
├───────────────────────────┼────────────┼───────────────────────────┼───────────────────────────┼────────────────┤
│ A method that lets you    │ Frequently │ ['This feature would be   │ As someone who works on   │ 5              │
│ include questions and     │            │ useful to me.', 'This     │ an open-source Python     │                │
│ responses as context for  │            │ feature would make me     │ package for conducting    │                │
│ new questions.            │            │ more productive.', 'This  │ research with AI, I think │                │
│                           │            │ feature will be important │ including questions and   │                │
│                           │            │ to me.', 'I would like to │ responses as context for  │                │
│                           │            │ see some examples of how  │ new questions could be    │                │
│                           │            │ to use this feature.']    │ incredibly valuable. Here │                │
│                           │            │                           │ are a few thoughts:       │                │
│                           │            │                           │                           │                │
│                           │            │                           │ 1. **Enhanced Contextual  │                │
│                           │            │                           │ Understanding**: By       │                │
│                           │            │              

In [12]:
# Post the notebook on the Coop
from edsl import Notebook

n = Notebook(path = "digital_twin.ipynb")

n.push(description="Digital Twin", visibility="public") 

{'description': 'Digital Twin',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/3fb6fb6f-0adf-4db1-a23c-193909162b2d',
 'uuid': '3fb6fb6f-0adf-4db1-a23c-193909162b2d',
 'version': '0.1.33.dev1',
 'visibility': 'public'}

In [15]:
# Update an object at the Coop
n = Notebook(path = "digital_twin.ipynb")

n.patch(uuid = "3fb6fb6f-0adf-4db1-a23c-193909162b2d", value = n) 

{'status': 'success'}